In [1]:
import docx2txt
import xlrd
import pandas as pd
import re 
import numpy as np
import os
from tqdm import tqdm
import sys
import copy

In [8]:
tmp_file_path = r"./tmp.txt"

In [4]:
excel = pd.read_excel(r"./data/decoration/manual_code.xls",header=None,sheet_name='Sheet1')
excel

,0,1,2,3,4,5,6
0,故障代码 [989L00] 发动机控制器锁紧警告 1,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,报警级别,故障代码,故障,发动机控制器锁定警告 1,NaN,NaN,NaN
3,―,989L00,NaN,（机器监控器系统）,NaN,NaN,NaN
4,故障内容,· 检测到发动机控制器锁定（因素 1）。,NaN,NaN,NaN,NaN,NaN
5,控制器的反应,· 无特别反应。,NaN,NaN,NaN,NaN,NaN
6,NaN,· 如果故障原因消失，系统自身重设。,NaN,NaN,NaN,NaN,NaN
7,机器发生的故障,· 发动机不起动。,NaN,NaN,NaN,NaN,NaN
8,相关信息,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN,NaN
9,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN,NaN


In [2]:
workbook = xlrd.open_workbook(r"./data/decoration/manual_code.xls",formatting_info=True)

sheet = workbook.sheet_by_index(0)

In [12]:
synonymMap = {'报警级别':['报警级别'], '故障代码':['故障代码'], '故障':['故障'], '故障内容':['故障内容'],  \
              '控制器的反应':['控制器的反应'], '机器发生的故障':['机器发生的故障'],  \
             '相关信息':['相关信息'], '起因':['起因'], '诊断操作':['诊断操作']}

name_to_en = {"故障代码":"faultCode", "故障名称":"faultName", "报警级别":"alertLevel","故障内容":"faultContent", 
                 "控制器的反应":"controllerReaction", "机器发生的故障":"machineFault", "相关信息":"relatedInfo", 
                 "起因":"cause", "诊断操作":"diagnosisAction", "标准值":"standardVal", "附图":"image", 
                  "故障部位系统":"monitorSystem", "故障":"faultName"}
name_to_en = {k: v.lower() for k, v in name_to_en.items()}
name_to_ch = {v: k for k, v in name_to_en.items()}
name_to_ch

{'faultcode': '故障代码',
 'faultname': '故障',
 'alertlevel': '报警级别',
 'faultcontent': '故障内容',
 'controllerreaction': '控制器的反应',
 'machinefault': '机器发生的故障',
 'relatedinfo': '相关信息',
 'cause': '起因',
 'diagnosisaction': '诊断操作',
 'standardval': '标准值',
 'image': '附图',
 'monitorsystem': '故障部位系统'}

In [5]:
code_line_num = []
for i in range(len(excel)):
#     print(i)
#     re.match(r'故障代码 \[\w*\].*',str(excel[i][0])) is not None:
    match_result = re.match(r'故障代码\s*\[([\w|@]*)\]\s*(.*)',str(excel.loc[i,0])) 
    if match_result is not None:
#         print(excel.loc[i,0])
#         print(match_result.group(1), match_result.group(2))
        code_line_num.append(i)
code_line_num.append(len(excel))
print(len(code_line_num))
code_line_num
        

128


[0,
 16,
 32,
 47,
 93,
 180,
 199,
 240,
 262,
 309,
 330,
 348,
 365,
 419,
 475,
 530,
 593,
 639,
 688,
 737,
 789,
 813,
 860,
 911,
 974,
 989,
 1007,
 1047,
 1100,
 1147,
 1202,
 1257,
 1312,
 1367,
 1422,
 1477,
 1494,
 1514,
 1553,
 1569,
 1615,
 1662,
 1700,
 1820,
 1836,
 1864,
 1919,
 1969,
 1988,
 2009,
 2061,
 2126,
 2162,
 2178,
 2290,
 2329,
 2367,
 2385,
 2399,
 2477,
 2607,
 2661,
 2749,
 2764,
 2784,
 2842,
 2909,
 3024,
 3131,
 3224,
 3300,
 3315,
 3331,
 3375,
 3394,
 3409,
 3431,
 3454,
 3482,
 3666,
 3716,
 3736,
 3806,
 3902,
 3970,
 4037,
 4107,
 4171,
 4240,
 4309,
 4372,
 4442,
 4511,
 4580,
 4649,
 4718,
 4735,
 4789,
 4833,
 4872,
 4913,
 5006,
 5104,
 5144,
 5204,
 5241,
 5281,
 5324,
 5365,
 5406,
 5446,
 5502,
 5556,
 5612,
 5666,
 5707,
 5747,
 5786,
 5826,
 5867,
 5908,
 5949,
 5990,
 6040,
 6090,
 6139,
 6189,
 6239]

## 逐个读取出每个实体/属性的值

In [10]:
fc_dicts = {}
for i_code in range(len(code_line_num)-1):
#     if i_code == 1:
#         break
    fc_dict = {}
    i_start = code_line_num[i_code]
    i_end = code_line_num[i_code+1]
#     print(str(excel.loc[i,0]))
    match_result = re.match(r'故障代码\s*\[([\w|@]*)\]\s*(.*)',str(excel.loc[i_start,0])) 
    fc_code = match_result.group(1)
    fc_content = match_result.group(2)
    print("i range in (%d,%d)"%(i_start, i_end))
#     print(excel.loc[i_start:i_end])

    for i in range(i_start, i_end):
        first_col_content = excel.loc[i,0]
        if first_col_content in synonymMap.keys():
#             print(first_col_content)
            if first_col_content == '报警级别':
                fc_dict[first_col_content] = excel.loc[i+1,0]
                for j in range(1,excel.shape[1]):
                    if excel.loc[i,j] == '故障代码':
#                         print(i+1)
#                         print(excel.loc[i+1,j])
                        fc_code_2 = re.match(r'\[?([\w|@]*)\]?',str(excel.loc[i+1,j])).group(1)
                        assert fc_code == fc_code_2, "%s %s"%(fc_code, fc_code_2)
                        fc_dict[excel.loc[i,j]] = fc_code
                    if excel.loc[i,j] == '故障':
                        if fc_content == excel.loc[i,j+1]:
                            fc_dict[excel.loc[i,j]] = fc_content
                        else:
                            print("WARNING：故障内容前后不一致：line_num=%d %s %s"%(i,fc_content, excel.loc[i,j+1]))
                            fc_dict[excel.loc[i,j]] = fc_content
#                         assert , 
                        
                        fc_dict['故障部位系统'] = excel.loc[i+1,j+1]
            if first_col_content == '故障内容':
#                 print(excel.shape)
#                 print(i_content)
#                 print(i)
#                 print(excel.loc[i_content,1])
                i_content = i 
                if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                    content = excel.loc[i,1]
                else:
                    content = excel.loc[i,2]
                fc_dict[first_col_content] = content
            if first_col_content == '控制器的反应':
                fc_dict[first_col_content] = [excel.loc[i,2]]
                for i_now in range(i+1,i+5):
                    if excel.loc[i_now,0] != '机器发生的故障':
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            content = excel.loc[i_now,1]
                        else:
                            content = excel.loc[i_now,2]
                        if content == content: # 检测是否为nan
                            fc_dict[first_col_content].append(content)
                    else:
                        break
            if first_col_content == '机器发生的故障':
                fc_dict[first_col_content] = [excel.loc[i,2]]
                for i_now in range(i+1,i+5):
                    if excel.loc[i_now+1,0] != '相关信息': # ‘相关信息’那几个字总是偏下一栏
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            content = excel.loc[i_now,1]
                        else:
                            content = excel.loc[i_now,2]
                        if content == content: # 检测是否为nan
                            fc_dict[first_col_content].append(content)
                    else:
                        break
            if first_col_content == '相关信息':
                
                for i_content_start in range(i,i-3,-1):
                    cell = sheet.cell(i_content_start,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
#                     print(dump_dic)
                    if dump_dic['bottom_line_style'] == 2:
                        break
                i_content_start = i_content_start + 1 # 往上多找了一行
                
                for i_content_end in range(i,i+5,1):
                    cell = sheet.cell(i_content_end,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
                    if dump_dic['bottom_line_style'] == 2:
                        break
#                 i_content_end += 1 # 
                
                fc_dict[first_col_content] = []
                for i_content in range(i_content_start,i_content_end+1): # +1是因为range函数会去掉后边界
                    if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                        content = excel.loc[i_content,1]
                    else:
                        content = excel.loc[i_content,2]
                    if content == content: # 检测是否为nan
                        fc_dict[first_col_content].append(content)
                        
            if first_col_content == '起因':
                split_line_num = [i+1]
                for i_content_end in range(i,i_end,1):
                    cell = sheet.cell(i_content_end,0)
                    fmt = workbook.xf_list[cell.xf_index]
                    file = open(tmp_file_path, mode='w')
                    fmt.dump(file)
                    file.close()
                    file = open(tmp_file_path,mode='r')
                    dump_dic = {}
                    for line in file.readlines():
                        match_result = re.match('(.*):\s(\d)', line)
                        if match_result is not None:
                            key = match_result.group(1).strip()
                            value = match_result.group(2)
                            dump_dic[key] = int(value)
                    file.close()
                    if dump_dic['bottom_line_style'] == 2:
                        split_line_num.append(i_content_end + 1)
                split_line_num.append(i_content_end)
                
                fc_dict[first_col_content] = []
                fc_dict['诊断操作'] = []
                for i_split in range(len(split_line_num)-1):
                    key = ""
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        if excel.loc[i_content,1] != '.' and excel.loc[i_content,1] != '·' and excel.loc[i_content,1] == excel.loc[i_content,1]:
                            key = excel.loc[i_content,1]
                            break
                    value = ""
#                     diagnosisAction = []
                    for i_content in range(split_line_num[i_split], split_line_num[i_split+1]):
                        for j in range(2,excel.shape[1]):
                            value_now = excel.loc[i_content,j]
                            if value_now != '.' and value_now != '·' and value_now == value_now:
                                value += str(value_now)
                    fc_dict[first_col_content].append(key)
                    fc_dict['诊断操作'].append(value)
                    
#     print(fc_dict)
    fc_dicts[fc_dict['故障代码']] = fc_dict
fc_dicts
    
    

i range in (0,16)
WARNING：故障内容前后不一致：line_num=2 发动机控制器锁紧警告 1 发动机控制器锁定警告 1
i range in (16,32)
WARNING：故障内容前后不一致：line_num=18 发动机控制器锁紧警告 2 发动机控制器锁定警告 2
i range in (32,47)
WARNING：故障内容前后不一致：line_num=34 发动机控制器锁紧警告 3 发动机控制器锁定警告 3
i range in (47,93)
i range in (93,180)
i range in (180,199)
i range in (199,240)
i range in (240,262)
i range in (262,309)
WARNING：故障内容前后不一致：line_num=264 散热器冷却液液位低 散热器冷却水水位低
i range in (309,330)
i range in (330,348)
WARNING：故障内容前后不一致：line_num=332 ECM 临界内部故障 发动机控制器内部故障
i range in (348,365)
i range in (365,419)
i range in (419,475)
i range in (475,530)
i range in (530,593)
i range in (593,639)
i range in (639,688)
i range in (688,737)
i range in (737,789)
i range in (789,813)
i range in (813,860)
i range in (860,911)
i range in (911,974)
i range in (974,989)
i range in (989,1007)
i range in (1007,1047)
i range in (1047,1100)
i range in (1100,1147)
i range in (1147,1202)
WARNING：故障内容前后不一致：line_num=1149 喷油器 #1（L#1）断路/短路故障 喷油器#1（L#1）断路/短路故障
i range in (1202,1257)
WARNING：

{'989L00': {'报警级别': '―',
  '故障代码': '989L00',
  '故障': '发动机控制器锁紧警告 1',
  '故障部位系统': '（机器监控器系统）',
  '故障内容': '· 检测到发动机控制器锁定（因素 1）。',
  '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'],
  '机器发生的故障': [nan,
   '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '相关信息': ['· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '起因': ['', '机器监控器的更换', ''],
  '诊断操作': ['', '此故障可能是由更换机器监控器引起。', '']},
 '989M00': {'报警级别': '―',
  '故障代码': '989M00',
  '故障': '发动机控制器锁紧警告 2',
  '故障部位系统': '（机器监控器系统）',
  '故障内容': '· 检测到发动机控制器锁定（因素 2）。',
  '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'],
  '机器发生的故障': [nan,
   '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '相关信息': ['· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。',
   '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'],
  '起因': ['', '机器监控器的更换', '机器监控器有问题', ''],
  '诊断操作': ['',
   '此故障可能是由更换机器监控器引起。',
   '当没有更换机器监控器时，显示此故障代码，机器监控器可能有问题。（由于 是内部故障，故不能进行故障诊断。）',
   '']},
 '989N00': {'报警级别': '―',
  '故障代码': '989N00',
  '故障': '发动机控制

## 将故障值从字典形式转化为关系表形式

In [13]:
merge = pd.DataFrame(columns=name_to_en.keys())
for fc_dict_oral in fc_dicts.values():
    print(fc_dict)
    fc_dict = copy.deepcopy(fc_dict_oral)
    list_keys = []
    for key in fc_dict.keys():
        print(type(fc_dict[key]))
        if isinstance(fc_dict[key], list):
            list_keys.append(key)
    fc_dict
    print(list_keys)
    for key in list_keys:
        fc_dict.pop(key, None)
    single_fc_df = pd.DataFrame(fc_dict, index=[0])
    
    fc_dict = copy.deepcopy(fc_dict_oral)
    # fc_dict_list = {}
    for key in list_keys:
        if key == '诊断操作':
            continue
        fc_dict_list = {}
#         print(fc_dict)
        fc_dict_list['故障代码'] = [fc_dict['故障代码']] * len(fc_dict[key])
        fc_dict_list[key] = fc_dict[key]
#         print(fc_dict_list)
#         print(pd.DataFrame(fc_dict_list))
        tmp_df = pd.DataFrame(fc_dict_list)
    #     a = pd.concat((a,b))
        single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    
    fc_dict = copy.deepcopy(fc_dict_oral)
    cause_dict = {}
    cause_dict['起因'] = []
    cause_dict['诊断操作'] = []
#     print(fc_dict)
    for i in range(len(fc_dict['起因'])):
        cause = fc_dict['起因'][i]
        diagnosisAction = fc_dict['诊断操作'][i]
        cause_dict['起因'].append(cause)
        cause_dict['诊断操作'].append(diagnosisAction)
#     print(cause_dict)
    if len(cause_dict) == 1:
        tmp_df = pd.DataFrame(cause_dict, index=[0])
    else:
        tmp_df = pd.DataFrame(cause_dict)

    single_fc_df = pd.concat((single_fc_df,tmp_df),sort=True )
    merge = pd.concat((merge,single_fc_df))
    print("*"*80)
merge

{'报警级别': '—', '故障代码': 'DY2EKB', '故障': '雨刷器（后退）短路', '故障部位系统': '（泵控制器系统）', '故障内容': '· 向雨刷器马达驱动的反转侧输出电流时，检测到有异常电流流动和短路。', '控制器的反应': [nan, '· 即使故障原因消失，在起动开关转到 OFF 位置之前，机器不能恢复正常。'], '机器发生的故障': [nan, '· 由于用于泵控制器接头的 T 形接头是“插座式盒”，所以无法在泵控制器接头处测量操作电压。', '· 重新生成故障代码的方法：将起动开关转到 ON 位置，并将雨刷器开关设定为 INT 或 ON。'], '相关信息': ['· 由于用于泵控制器接头的 T 形接头是“插座式盒”，所以无法在泵控制器接头处测量操作电压。', '· 重新生成故障代码的方法：将起动开关转到 ON 位置，并将雨刷器开关设定为 INT 或 ON。'], '起因': ['', '雨刷器马达故障', '线束短路', '线束接地故障', '泵控制器故障', ''], '诊断操作': ['', '1.  将起动开关转到 OFF 位置。2.  断开接头 M05，并将 T 形接头连接到凹侧。在 M05（凹）（1）和（3）之间导通在 M05（凹）（1）和接地之间电阻最小 1 MΩ', '1.  将起动开关转到 OFF 位置。2.  断开接头 C02 和 M05，并将 T 形接头连接到它们任意之一的线束侧。在 C02（凹）（119）和（114）之间，或 M05（凸）电阻最小 1 MΩ（1）和（3）之间', '1.  将起动开关转到 OFF 位置。2.  断开接头 C02 和 M05，并将 T 形接头连接到它们任意之一的线束侧。在 C02（凹）（114）或 M05（凹）（1）和接地之 间电阻最小 1 MΩ', '如果在上述检查中未发现故障，则泵控制器故障。（由于这是内部故障，所以无法进行故障诊断。）· 参考1.  将起动开关转到 OFF 位置。2.  将 T 形接头插入接头 M05。3.  将起动开关转到 ON 位置。4.  打开和关闭雨刷器开关，并进行故障诊断。雨刷器开关：电压最高 3 V在 M05（1）和（5） 之间OFF雨刷器开关：电压脉冲低于 3V ←→ 高于ON20 V', '']}
<class 'str'>
<cla

C:\Users\Yaoyao\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



********************************************************************************
{'报警级别': '―', '故障代码': '989L00', '故障': '发动机控制器锁紧警告 1', '故障部位系统': '（机器监控器系统）', '故障内容': '· 检测到发动机控制器锁定（因素 1）。', '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'], '机器发生的故障': [nan, '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。', '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'], '相关信息': ['· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。', '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'], '起因': ['', '机器监控器的更换', ''], '诊断操作': ['', '此故障可能是由更换机器监控器引起。', '']}
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
['控制器的反应', '机器发生的故障', '相关信息', '起因', '诊断操作']
********************************************************************************
{'报警级别': '―', '故障代码': '989M00', '故障': '发动机控制器锁紧警告 2', '故障部位系统': '（机器监控器系统）', '故障内容': '· 检测到发动机控制器锁定（因素 2）。', '控制器的反应': [nan, '· 如果故障原因消失，系统自身重设。'], '机器发生的故障': [nan, '· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。', '· 重新生成故障代码的方法: 将起动开关转到 ON 位置。'], '相关信息': ['· 如果在

{'报警级别': 'L01', '故障代码': 'B@HANS', '故障': '液压油过热', '故障部位系统': '（泵控制器系统）', '故障内容': '当发动机运转时，液压油温度传感器的信号电压检测到液压油过热（102 °C 以上）。', '控制器的反应': ['在机器监控器上显示液压油温度监测画面红色。 如果故障原因消失，机器自身变为正常。'], '机器发生的故障': ['如果照旧操作机器，液压部件会损坏。'], '相关信息': ['将液压油温度传感器信号输入泵控制器，然后通过 CAN 通讯系统将其传送到机器监控器。', '可以用监测功能检查液压油温度。（代码：04401 液压油温度）', '见 “液压油温度计不正常工作” E 模式 E-9 的故障诊断。', '重新生成故障代码的方法：起动发动机。'], '起因': ['', '液压油过热', '液压油温度测量系统不良', ''], '诊断操作': ['', '液压油可能过热。如果过热，进行检查并排除原因。', '如果没有检测到原因 1，液压油温监控系统可能有问题。进行故障代码[DGH2KB]的故障诊断。', '']}
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
['控制器的反应', '机器发生的故障', '相关信息', '起因', '诊断操作']
********************************************************************************
{'报警级别': 'L04', '故障代码': 'CA111', '故障': 'ECM 临界内部故障', '故障部位系统': '（发动机控制器系统）', '故障内容': '发动机控制器内记忆故障或电源电路故障。', '控制器的反应': ['无特殊反应'], '机器发生的故障': ['发动机在操作中会停机或可能无法起动。'], '相关信息': ['可以用监控功能检查至发动机控制器的电源电压。', '（代码：03203 蓄电池电压）', '重新生成故障代码的方法：起

<class 'str'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
['控制器的反应', '机器发生的故障', '相关信息', '起因', '诊断操作']
********************************************************************************
{'报警级别': 'L01', '故障代码': 'CA145', '故障': '冷却液温度传感器低压故障', '故障部位系统': '（发动机控制器系统）', '故障内容': '冷却液温度传感器的信号电路内出现低压。', '控制器的反应': ['将冷却液温度设定在固定值，并运转发动机。'], '机器发生的故障': ['发动机在低温状态下不起动。', '防止过热功能不工作。', '可以用监控功能检查来自发动机冷却液温度传感器的输入电压。'], '相关信息': ['（代码：04105（V））', '可以用监控功能检查发动机冷却液温度传感器感应到的温度。', '（代码：04107（°C））', '重新生成故障代码的方法：将起动开关转到 ON 位置。', '如果温度传感器断开，则不显示此故障代码，但是显示“高压故障”的故障代码[CA144]。'], '起因': ['', '线束接头故障', '冷却液温度传感器故障', '线束短路', '线束接地故障', '发动机控制器故障', ''], '诊断操作': ['', '见“故障诊断概述”中“故障诊断前的检查”的“c：电气装置”中的线束和 接头内容，然后对其进行直接检查。', '1.  将起动开关转到 OFF 位置。2.  断开接头 COOLANT TEMP，并将插座连接到凸侧。★ 如果冷却液温度在 0 °C 以上时，冷却液温度传感器电阻为 700 Ω-37 kΩ， 则将冷却液温度传感器视为正常。0 °C电阻30 至 37 kΩ在 COOLANT TEMP（凸）25 °C电阻9.3 至 10.7 kΩ（A）和（B）之间50 °C电阻3.2 至 3.8 kΩ★ 冷却液温度-电阻特性80 °C电阻1.0 至 1.3 kΩ95 °C电阻700 至 800 Ω在（B）和接地之间冷却液温度电阻最小 100 kΩ的整个范围'

...

{'报警级别': 'L03', '故障代码': 'DW91KA', '故障': '行走连通电磁阀断路', '故障部位系统': '（泵控制器系统）', '故障内容': '泵控制器不驱动行走连通电磁阀时，控制器至电磁阀的输出电压高于标准值，所以泵控制器确定', '控制器的反应': ['无特殊反应（由于没有电流流动，所以电磁阀未通电。）', '如果故障原因消失，则机器自己恢复正常。'], '机器发生的故障': ['机器在行走时不能转向。'], '相关信息': ['可以用监控功能检查至行走连通电磁阀的控制器指令（ON/OFF）。（如果控制器发出电磁阀 ON', '指令，则监控显示为 ON，即使电磁阀由于断路没有打开。）（代码：02300 1 号电磁阀）', '重新生成故障代码的方法：起动发动机并用左右行走操纵杆使机器转向。'], '起因': ['', '行走连通电磁阀故障', '线束断路或短路', '线束断路', '泵控制器故障', ''], '诊断操作': ['', '1.  将起动开关转到 OFF 位置。2.  断开接头 V05，并将 T 形接头连接到凸侧。在 V05（凸）（1）和（2）之间电阻20 至 60 Ω', '1.  将起动开关转到 OFF 位置。2.  断开接头 C02，并将 T 形接头连接到凹侧。在 C02（凹）（110）和（115）、（117）以及（120） 之间电阻20 至 60 Ω', '★ 如果在原因 2 的检查中未发现故障，则不需要进行此检查。1.  将起动开关转到 OFF 位置。2.  断开接头 C02 和 V05，并将 T 形接头连接到各凹侧。在 C02（凹）（110）和 V05（凹）（1）之间电阻最大 1 Ω在 V05（凹）（2）和 C02（凹）（115）或（117） 或（120）之间电阻最大 1 Ω', '如果在上述检查中未发现故障，则泵控制器故障。（由于这是内部故障，所以无法进行故障诊断。）', '']}
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
['控制器的反应', '机器发生的故障', '

{'报警级别': 'L01', '故障代码': 'DWK2KB', '故障': '可变背压电磁阀短路', '故障部位系统': '（泵控制器系统）', '故障内容': '泵控制器驱动可变背压电磁阀时，电流流动异常，所以泵控制器确定可变背压电磁阀短路。', '控制器的反应': ['停止驱动可变背压电磁阀。', '即使故障原因消失，在起动开关转到 OFF 位置之前，机器不能恢复正常。'], '机器发生的故障': ['背压值的设定压力不能改变。'], '相关信息': ['可以用监控功能检查至可变背压电磁阀的控制器指令（ON/OFF）。（如果控制器发出电磁阀 ON', '指令，则监控显示为 ON，即使电磁阀由于短路没有打开。）（代码：02301 2 号电磁阀）', '电磁阀打开时，背压值的设定压力变低。', '重新生成故障代码的方法：将起动开关转到 ON 位置。'], '起因': ['', '背压电磁阀故障', '线束断路或接地故障', '线束接地故障', '泵控制器故障', ''], '诊断操作': ['', '1.  将起动开关转到 OFF 位置。2.  断开接头 V25，并将 T 形接头连接到凸侧。在 V25（凸）（1）和（2）之间电阻20 至 60 Ω在 V25（凸）（1）和接地之间电阻最小 1 MΩ', '1.  将起动开关转到 OFF 位置。2.  断开接头 C02，并将 T 形接头连接到凹侧。在 C02（凹）（85）和（115）、（117）以及（120） 之间电阻20 至 60 Ω', '★ 如果在原因 2 的检查中未发现故障，则不需要进行此检查。1.  将起动开关转到 OFF 位置。2.  断开接头 C02 和 V25，并将 T 形接头连接到各凹侧。在接地和 C02（凹）（85）或 V25（凹）（1）之间电阻最小 1 MΩ', '如果在上述检查中未发现故障，则泵控制器故障。（由于这是内部故障，所以无法进行故障诊断。）', '']}
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
['控制器的反应', '机器发生的故障', 

{'报警级别': 'L01', '故障代码': 'DXE0KB', '故障': 'LS-EPC 电磁线圈短路', '故障部位系统': '（泵控制器系统）', '故障内容': '驱动 LS-EPC 电磁线圈时，检测到有异常电流流动并且短路。', '控制器的反应': ['停止驱动 LS-EPC 电磁线圈。', '即使故障原因消失，在起动开关转到 OFF 位置之前，机器不能恢复正常。'], '机器发生的故障': ['将速度设定在 Lo（低速）和 Mi（中速）时，行走速度高。', '精控模式（L）中，工作装置速度和回转速度高。'], '相关信息': ['可以用监控功能检查 LS-EPC 电磁线圈的驱动电流。', '（代码：01500 LS-EPC 电磁线圈电流）', '重新生成故障代码的方法：将起动开关转到 ON 位置。'], '起因': ['', 'LS-EPC 电磁线圈故障', '线束短路或接地故障', '线束接地故障', '泵控制器故障', ''], '诊断操作': ['', '1.  将起动开关转到 OFF 位置。2.  断开接头 V19，并将 T 形接头连接到凸侧。在 V19（凸）（1）和（2）之间电阻3 至 14 Ω在 V19（凸）（2）和接地之间电阻最小 1 MΩ', '1.  将起动开关转到 OFF 位置。2.  断开接头 C02，并将 T 形接头连接到凹侧。在 C02（凹）（88）和（115）、（117）以及（120） 之间电阻3 至 14 Ω在 C02（凹）（88）和接地之间电阻最小 1 MΩ', '★ 如果在原因 2 的检查中未发现故障，则不需要进行此检查。1.  将起动开关转到 OFF 位置。2.  断开接头 C01 和 V19，并将 T 形接头连接到各凹侧。在 C02（凹）（88）或 V19（凹）（1）和接地之间电阻最小 1 MΩ', '如果在上述检查中未发现故障，则泵控制器故障。（由于这是内部故障，所以 无法进行故障诊断。）', '']}
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
['控制器的反应', 

{'报警级别': '—', '故障代码': 'DY20KA', '故障': '雨刷器工作异常', '故障部位系统': '（泵控制器系统）', '故障内容': '风挡雨刷器工作时，未输入工作端的 W 信号。', '控制器的反应': ['关闭雨刷器马达的电源。 如果故障原因消失，则机器自己恢复正常。'], '机器发生的故障': ['雨刷器马达不工作。'], '相关信息': ['（代码：02204 开关输入 5）', '· 由于用于泵控制器接头的 T 形接头是“插座式盒”，所以无法在泵控制器接头处测量操作电压。', '★ 单雨刷器没有双雨刷器的下部雨刷器（上部雨刷器称为雨刷器）。', '· 重新生成故障代码的方法：将起动开关转到 ON 位置，并将雨刷器开关设定为 INT 或 ON。'], '起因': ['', '雨刷器马达故障', '线束断路或短路', '线束断路', '泵控制器故障', ''], '诊断操作': ['', '1.  将起动开关转到 OFF 位置。2.  断开接头 M05，并将 T 形接头连接到凹侧。雨刷器位置：工作 区域端电阻最大 1 Ω在 M05（凹）（6）和（5）之 间雨刷器位置：工作 区域端之外的位置电阻最小 1 MΩ在 M05（凹）（1）和（3）之 间雨刷器位置：全部 范围电阻最大 20 Ω', '1.  将起动开关转到 OFF 位置。2.  断开接头 C01 和 C02，并将 T 形接头连接到各凹侧。雨刷器位置：工作 区域端电阻最大 1 Ω在 C01（凹）（56）和接地之 间雨刷器位置：工作 区域端之外的位置电阻最小 1 MΩ在 C02（凹）（114）和（119） 之间雨刷器位置：全部 范围电阻最大 20 Ω', '★ 如果在原因 2 的检查中未发现故障，则不需要进行此检查。1.  将起动开关转到 OFF 位置。2.  断开接头 CP01、CP02 和 M05。3.  将 T 形接头连接到接头 CP01 和 CP02 的凹侧和 M05 的凸侧。在 C01（凹）（56）和 M05（凸）（6）之间电阻最大 1 Ω在 M05（凸）（5）和接地（T05）之间电阻最大 1 Ω在 C02（凹）（114）和 M05（凸）（1）之间电阻最大 1 Ω在 C02（凹）（119）和 M05（凸）（3）之间电阻最大 1 Ω', '如果在上述检查中未发现故障，则泵控制器

,报警级别,控制器的反应,故障,故障代码,故障内容,故障名称,故障部位系统,机器发生的故障,标准值,相关信息,诊断操作,起因,附图
0,―,NaN,发动机控制器锁紧警告 1,989L00,· 检测到发动机控制器锁定（因素 1）。,NaN,（机器监控器系统）,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,· 如果故障原因消失，系统自身重设。,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,989L00,NaN,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,机器监控器的更换,NaN


In [14]:
merge = merge.rename(columns=name_to_en)
merge.to_csv("manual_fc.csv",quoting=1,sep=';',index=False,encoding='utf-16')
merge

,alertlevel,controllerreaction,faultname,faultcode,faultcontent,faultname,monitorsystem,machinefault,standardval,relatedinfo,diagnosisaction,cause,image
0,―,NaN,发动机控制器锁紧警告 1,989L00,· 检测到发动机控制器锁定（因素 1）。,NaN,（机器监控器系统）,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,· 如果故障原因消失，系统自身重设。,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,989L00,NaN,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,· 如果在更换机器监控器后，显示此故障代码，用户密码必须换回更换之前的密码。,NaN,NaN,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,· 重新生成故障代码的方法: 将起动开关转到 ON 位置。,NaN,NaN,NaN
0,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
1,NaN,NaN,NaN,989L00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,机器监控器的更换,NaN
